# 동적 페이지 크롤링하기(2)

동적 페이지로만 짜여진 페이지는 전용 페이지 url을 찾아야한다.   
예시로 맛집 정보 공유 사이트인 블루리본서베이 사이트 크롤링을 해보도록 하자.

disable JS 를 통해 JS임을 확인했다.  
### 자바스크립트의 URL을 찾아야 한다.  



### JavaScript URL 찾기
### Network탭에 들어가면 자바스크립트 페이지의 URL을 알아낼 수 있다.
1. network 탭으로 이동(clear - 새로고침)
2. XHR, JS 의 후보들을 하나씩 확인(preview)
    - 페이지(왼쪽)과 preview(오른쪽, json) 대조
3. 해당 요소를 찾았다면, 
   headers(preview 옆)로 이동 -> General - Requests URL


그렇다. 그 url이 우리가 원하던 URL이다.

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas 
import json

In [ ]:
url = 'https://www.bluer.co.kr/api/v1/restaurants?page=0&size=30&query=&foodType=&foodTypeDetail=&feature=&location=%EA%B2%BD%EA%B8%B0%EB%8F%84&locationDetail=&area=&areaDetail=&priceRange=&ribbonType=&recommended=false&isSearchName=false&searchMode=ribbonType&zone1=&zone2=&zone2Lat=&zone2Lng='

resp = requests.get(url)
resp

<Response [200]>

In [ ]:
# soup = BeautifulSoup(resp.content, 'lxml')
# soup.select()

### soup.select를 쓸 수 없다!! HTML이 아니라 JSON이라서..

page -> JSON

HTML  vs  JSON

둘다 page 데이터 

형식의 차이가 있다.

HTML : tag의 계층구조 -> BeautifulSoup, select(수식어)

JSON : dic 의 계층구조 -> 필요 X, dictionary, key


4) JSON은 딕셔너리로 변환한다.  
5) 딕셔너리에서 key값을 이용해 원하는 정보들을 파싱한다.



In [ ]:
# JSON 과 딕셔너리의 차이
# JSON : 딕셔너리처럼 생긴 문자열
# {'사과' : 1000} -> 이건 딕셔너리
# "{'사과' : 1000}" -> 이건 JSON

In [ ]:
# 딕셔너리의 계층구조인 JSON 파싱 연습
# 딕셔너리 dic 에서 '사과' 파싱해보기
dic = {'a' : {
            'b' : {
                'C' : {
                    'd' : '사과'
                    }
                }
            }
}

# a b C d
dic['a']['b']['C']['d']

'사과'

In [ ]:
# 가보정, 수원, 한와담, 서울 파싱해보기

dic = {'a' : {
            'b' : {
                'restaurants' : [{
                    'name' : '가보정',
                    'adress' : '수원'
                    },
                    {'name' : '한와담',
                    'adress' : '서울'
                    }]
                }
            }
}

# a b restaurants [] name, adress
restaurants = dic['a']['b']['restaurants']

for restaurant in restaurants:
    print(restaurant['name'])
    print(restaurant['adress'])

가보정
수원
한와담
서울


### JSON을 dict로 형변환하기

In [ ]:
dic = {'a' : {
            'b' : {
                'restaurants' : [{
                    'name' : '가보정',
                    'adress' : '수원'
                    },
                    {'name' : '한와담',
                    'adress' : '서울'
                    }]
                }
            }
        }

# dic['a'] # json은 key 파싱이 안됨 -> 딕셔너리로 형변환해야한다.
import json

jsn = json.dumps(dic) # dumps : dic -> json
jsn

'{"a": {"b": {"restaurants": [{"name": "\\uac00\\ubcf4\\uc815", "adress": "\\uc218\\uc6d0"}, {"name": "\\ud55c\\uc640\\ub2f4", "adress": "\\uc11c\\uc6b8"}]}}}'

In [ ]:
dic = json.loads(jsn) # loads : json -> dic
dic

{'a': {'b': {'restaurants': [{'adress': '수원', 'name': '가보정'},
    {'adress': '서울', 'name': '한와담'}]}}}

In [ ]:
import json 

bluers = {
    '가게명' : [],
    '리본등급' : [],
    "사이트" : [], 
    "전화번호": [], 
    "휴무": [],
    "주차":[],
    "가격범위":[],
    "메뉴":[],
    "영업시간":[]

}

dic_resp = json.loads(resp.text)
restaurants = dic_resp["_embedded"]["restaurants"]

for restaurant in restaurants:
    bluers['가게명'].append(restaurant["headerInfo"]["nameKR"])
    bluers['리본등급'].append(restaurant["headerInfo"]["ribbonType"])
    bluers["사이트"].append(restaurant["defaultInfo"]["website"])
    bluers["전화번호"].append(restaurant["defaultInfo"][ "phone"])
    bluers["휴무"].append(restaurant["defaultInfo"]["dayOff"])
    bluers["주차"].append(restaurant["statusInfo"]["parking"])
    bluers["가격범위"].append(restaurant["statusInfo"]["priceRange"])
    bluers["메뉴"].append(restaurant["statusInfo"]["menu"])
    bluers["영업시간"].append(restaurant["statusInfo"]["businessHours"])


df_bluer=pd.DataFrame(bluers)
df_bluer    

,가게명,리본등급,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,가보정,RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,연중무휴,가능,2~4만원대,"한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g...","11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00"
1,강천매운탕,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,연중무휴,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...",10:00~22:00
2,고기리막국수,RIBBON_TWO,http://고기리막국수.com/,031-263-1107,화요일 휴무,가능,1만원 미만,"물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원)",11:00~21:00(마지막 주문 20:30)
3,기와집순두부,RIBBON_TWO,,031-576-9009,명절 휴무,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천...",하절기 10:00~21:00 | 동절기 10:30~21:00
4,능라도,RIBBON_TWO,,031-781-3989,명절 당일 휴무,가능,1~2만원대,"평양냉면, 평양온면(각 1만4천원), 평양온반(1만2천원), 접시만두(각 1만3천원...",11:00~21:00(마지막 주문 20:30)
5,명월집,RIBBON_TWO,,031-584-4111,"월, 화, 수요일 휴무",가능,2~4만원대,돼지불고기(250g 3만원),16:00~19:30
6,사각하늘,RIBBON_TWO,,031-774-3670,"월, 화요일 휴무",가능,4~10만원대,"점심스키야키(2인 이상, 1인 4만3천원), 저녁스키야키(2인 이상, 1인 5만5천원)",12:00~22:00
7,상해루,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,월요일 휴무,가능,1~2만원대,"삼선짜장(8천5백원), 짬뽕(8천원) , 탕수육(소 2만원, 중 3만원), 오향장육...",11:00~15:00/17:00~22:00 | 주말 11:00~22:00
8,서동관,RIBBON_TWO,,031-922-7463,"화요일, 명절 휴무",가능,1~2만원대,"곰탕(1만3천원), 양곰탕(1만5천원), 특곰탕(1만7천원), 이공탕(2만원), 이...",09:00~21:00(마지막 주문 20:30)
9,세야스시,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,월요일 휴무,가능,10~20만원대,"런치오마카세(13만원), 디너오마카세(23만원), 콜키지(5만원)",12:00~15:00/18:00~22:00


In [ ]:
# "_embedded" "restaurants": [] "headerInfo" "nameKR", "ribbonType"
# "_embedded" "restaurants": [] "defaultInfo" "website", "phone", "dayOff"
# "_embedded" "restaurants": [] "statusInfo" "parking", "visit", "menu", "priceRange", "businessHours"
# "_embedded" "restaurants": [] "juso" "roadAddrPart1"

In [ ]:
# 조건 검색
df_bluer[df_bluer['주차'] == '가능']

,가게명,리본등급,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,가보정,RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,연중무휴,가능,2~4만원대,"한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g...","11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00"
1,강천매운탕,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,연중무휴,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...",10:00~22:00
2,고기리막국수,RIBBON_TWO,http://고기리막국수.com/,031-263-1107,화요일 휴무,가능,1만원 미만,"물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원)",11:00~21:00(마지막 주문 20:30)
3,기와집순두부,RIBBON_TWO,,031-576-9009,명절 휴무,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천...",하절기 10:00~21:00 | 동절기 10:30~21:00
4,능라도,RIBBON_TWO,,031-781-3989,명절 당일 휴무,가능,1~2만원대,"평양냉면, 평양온면(각 1만4천원), 평양온반(1만2천원), 접시만두(각 1만3천원...",11:00~21:00(마지막 주문 20:30)
5,명월집,RIBBON_TWO,,031-584-4111,"월, 화, 수요일 휴무",가능,2~4만원대,돼지불고기(250g 3만원),16:00~19:30
6,사각하늘,RIBBON_TWO,,031-774-3670,"월, 화요일 휴무",가능,4~10만원대,"점심스키야키(2인 이상, 1인 4만3천원), 저녁스키야키(2인 이상, 1인 5만5천원)",12:00~22:00
7,상해루,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,월요일 휴무,가능,1~2만원대,"삼선짜장(8천5백원), 짬뽕(8천원) , 탕수육(소 2만원, 중 3만원), 오향장육...",11:00~15:00/17:00~22:00 | 주말 11:00~22:00
8,서동관,RIBBON_TWO,,031-922-7463,"화요일, 명절 휴무",가능,1~2만원대,"곰탕(1만3천원), 양곰탕(1만5천원), 특곰탕(1만7천원), 이공탕(2만원), 이...",09:00~21:00(마지막 주문 20:30)
9,세야스시,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,월요일 휴무,가능,10~20만원대,"런치오마카세(13만원), 디너오마카세(23만원), 콜키지(5만원)",12:00~15:00/18:00~22:00


In [ ]:
cols = list(df_bluer)

df_bluer[cols[:3]]

,가게명,리본등급,사이트
0,가보정,RIBBON_TWO,http://www.kabojong.co.kr
1,강천매운탕,RIBBON_TWO,http://www.365food.com/gang
2,고기리막국수,RIBBON_TWO,http://고기리막국수.com/
3,기와집순두부,RIBBON_TWO,
4,능라도,RIBBON_TWO,
5,명월집,RIBBON_TWO,
6,사각하늘,RIBBON_TWO,
7,상해루,RIBBON_TWO,http://appweb.co.kr/shanghairoo
8,서동관,RIBBON_TWO,
9,세야스시,RIBBON_TWO,http://seyasushi.modoo.at/


In [ ]:
# 딕셔너리에서 list 형변환하면 결과가 키값 리스트
# 데이터프레임을 list 형변환하면 결과가 컬럼명 리스트
cols = list(df_bluer)

cols.insert(0, cols.pop(1))

df_bluer = df_bluer[cols]
df_bluer

,리본등급,가게명,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,RIBBON_TWO,가보정,http://www.kabojong.co.kr,1600-3883,연중무휴,가능,2~4만원대,"한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g...","11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00"
1,RIBBON_TWO,강천매운탕,http://www.365food.com/gang,031-882-5191,연중무휴,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...",10:00~22:00
2,RIBBON_TWO,고기리막국수,http://고기리막국수.com/,031-263-1107,화요일 휴무,가능,1만원 미만,"물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원)",11:00~21:00(마지막 주문 20:30)
3,RIBBON_TWO,기와집순두부,,031-576-9009,명절 휴무,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천...",하절기 10:00~21:00 | 동절기 10:30~21:00
4,RIBBON_TWO,능라도,,031-781-3989,명절 당일 휴무,가능,1~2만원대,"평양냉면, 평양온면(각 1만4천원), 평양온반(1만2천원), 접시만두(각 1만3천원...",11:00~21:00(마지막 주문 20:30)
5,RIBBON_TWO,명월집,,031-584-4111,"월, 화, 수요일 휴무",가능,2~4만원대,돼지불고기(250g 3만원),16:00~19:30
6,RIBBON_TWO,사각하늘,,031-774-3670,"월, 화요일 휴무",가능,4~10만원대,"점심스키야키(2인 이상, 1인 4만3천원), 저녁스키야키(2인 이상, 1인 5만5천원)",12:00~22:00
7,RIBBON_TWO,상해루,http://appweb.co.kr/shanghairoo,031-8015-0103,월요일 휴무,가능,1~2만원대,"삼선짜장(8천5백원), 짬뽕(8천원) , 탕수육(소 2만원, 중 3만원), 오향장육...",11:00~15:00/17:00~22:00 | 주말 11:00~22:00
8,RIBBON_TWO,서동관,,031-922-7463,"화요일, 명절 휴무",가능,1~2만원대,"곰탕(1만3천원), 양곰탕(1만5천원), 특곰탕(1만7천원), 이공탕(2만원), 이...",09:00~21:00(마지막 주문 20:30)
9,RIBBON_TWO,세야스시,http://seyasushi.modoo.at/,031-8015-2216,월요일 휴무,가능,10~20만원대,"런치오마카세(13만원), 디너오마카세(23만원), 콜키지(5만원)",12:00~15:00/18:00~22:00


In [ ]:
df_bluer.columns

Index(['가게명', '리본등급', '사이트', '전화번호', '휴무', '주차', '가격범위', '메뉴', '영업시간'], dtype='object')

In [ ]:
df_bluer.index

RangeIndex(start=0, stop=30, step=1)

In [ ]:
df_bluer.values

array([['가보정', 'RIBBON_TWO', 'http://www.kabojong.co.kr', '1600-3883',
        '연중무휴', '가능', '2~4만원대',
        '한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g 6만2천원), 육회(180g 3만7천원), 미국산생갈비(450g 4만9천원), 미국산양념갈비(450g 4만6천원)',
        '11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00 '],
       ['강천매운탕', 'RIBBON_TWO', 'http://www.365food.com/gang',
        '031-882-5191', '연중무휴', '가능', '2~4만원대',
        '빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원), 메기+빠가사리매운탕(소 4만원, 중 5만원, 대 6만원, 특 7만원), 자연산장어구이(1kg 16만원), 쏘가리회(1kg 15만원), 용봉탕(시가) ',
        '10:00~22:00 '],
       ['고기리막국수', 'RIBBON_TWO', 'http://고기리막국수.com/', '031-263-1107',
        '화요일 휴무', '가능', '1만원 미만',
        '물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원) ',
        '11:00~21:00(마지막 주문 20:30) '],
       ['기와집순두부', 'RIBBON_TWO', '', '031-576-9009', '명절 휴무', '가능',
        '1만원 미만',
        '순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천원), 두부김치, 파전, 녹두전, 도토리묵, 군두부(각 1만4천원), 생두부&수육(2만4천원), 수육(2만9천원) ',
        '하절기 10:00~21:00 | 동절기 10:30

In [ ]:
# dataframe의 컬럼 순서 조정 -> 컬럼명 리스트로 조정하는게 편하다
df_bluer = df_bluer[['가게명','영업시간', '리본등급', '사이트', '전화번호',  '주차', '가격범위', '메뉴', '휴무']]
df_bluer

,가게명,영업시간,리본등급,사이트,전화번호,주차,가격범위,메뉴,휴무
0,가보정,"11:30~22:00 | 주말, 공휴일 11:00~22:00",RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,가능,2~4만원대,"한우생갈비(250g 7만2천원), 양념갈비(270g 5만3천원), 채끝등심(150g...",연중무휴
1,강천매운탕,10:00~22:00,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...","첫째, 셋째 주 화요일 휴무"
2,고기리막국수,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-263-1107,가능,1만원 미만,"비빔막국수, 들기름막국수(각 8천원), 수육(소 1만3천원, 중 1만9천원)",화요일 휴무
3,기와집순두부,하절기 10:00~21:00 | 동절기 10:30~21:00,RIBBON_TWO,,031-576-9009,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 8천원), 재래식생두부, 황태양념구이(각 1만원)...",명절 휴무
4,능라도,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-781-3989,가능,1~2만원대,"평양냉면, 평양온면(각 1만2천원), 평양온반, 접시만두(각 1만1천원), 제육(2...",명절 당일 휴무
5,명월집,16:00~19:30,RIBBON_TWO,,031-584-4111,불가,2~4만원대,돼지불고기(250g 2만원),"월, 화, 수요일 휴무"
6,백제장,11:00~21:00,RIBBON_TWO,,031-746-4296,가능,2~4만원대,"산채정식(2인 이상, 1인 2만1천원), 숯불불고기(200g 1만8천원), 녹두빈대...",연중무휴
7,사각하늘,12:00~22:00,RIBBON_TWO,,031-774-3670,가능,4~10만원대,"점심스키야키(2인 이상, 1인 3만8천원), 저녁스키야키(2인 이상, 1인 5만5천원)","월요일, 화요일 휴무"
8,상해루,11:00~15:00/17:00~22:00 | 주말 11:00~22:00,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,가능,1~2만원대,"탕수육(소 1만8천원, 대 2만7천원), 멘보샤(2만7천원), 오향장육(3만3천원)...",월요일 휴무
9,세야스시,12:00~15:00/18:00~22:00,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,가능,10~20만원대,"런치오마카세(10만원), 디너오마카세(20만원)",월요일 휴무


In [ ]:
# 삼추사삭 = 리스트의 3가지의 추가 메서드, 4가지의 삭제 메서드
# a - append
# i - insert
# e - extend

# r - remove
# p - pop
# c - clear
# d - del

# ['가게명','영업시간', '리본등급', '사이트', '전화번호',  '주차', '가격범위', '메뉴', '휴무']
# -> ['가게명', '가격범위','영업시간', '리본등급', '사이트', '전화번호',  '주차', '메뉴', '휴무']

In [ ]:
cols = list(df_bluer.columns)

In [ ]:
cols.insert(1, cols.pop(-3))
cols

['가게명', '가격범위', '영업시간', '리본등급', '사이트', '전화번호', '주차', '메뉴', '휴무']

In [ ]:
df_bluer[cols]

,가게명,가격범위,영업시간,리본등급,사이트,전화번호,주차,메뉴,휴무
0,가보정,2~4만원대,"11:30~22:00 | 주말, 공휴일 11:00~22:00",RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,가능,"한우생갈비(250g 7만2천원), 양념갈비(270g 5만3천원), 채끝등심(150g...",연중무휴
1,강천매운탕,2~4만원대,10:00~22:00,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,가능,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...","첫째, 셋째 주 화요일 휴무"
2,고기리막국수,1만원 미만,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-263-1107,가능,"비빔막국수, 들기름막국수(각 8천원), 수육(소 1만3천원, 중 1만9천원)",화요일 휴무
3,기와집순두부,1만원 미만,하절기 10:00~21:00 | 동절기 10:30~21:00,RIBBON_TWO,,031-576-9009,가능,"순두부백반, 콩탕백반, 비빔밥(각 8천원), 재래식생두부, 황태양념구이(각 1만원)...",명절 휴무
4,능라도,1~2만원대,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-781-3989,가능,"평양냉면, 평양온면(각 1만2천원), 평양온반, 접시만두(각 1만1천원), 제육(2...",명절 당일 휴무
5,명월집,2~4만원대,16:00~19:30,RIBBON_TWO,,031-584-4111,불가,돼지불고기(250g 2만원),"월, 화, 수요일 휴무"
6,백제장,2~4만원대,11:00~21:00,RIBBON_TWO,,031-746-4296,가능,"산채정식(2인 이상, 1인 2만1천원), 숯불불고기(200g 1만8천원), 녹두빈대...",연중무휴
7,사각하늘,4~10만원대,12:00~22:00,RIBBON_TWO,,031-774-3670,가능,"점심스키야키(2인 이상, 1인 3만8천원), 저녁스키야키(2인 이상, 1인 5만5천원)","월요일, 화요일 휴무"
8,상해루,1~2만원대,11:00~15:00/17:00~22:00 | 주말 11:00~22:00,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,가능,"탕수육(소 1만8천원, 대 2만7천원), 멘보샤(2만7천원), 오향장육(3만3천원)...",월요일 휴무
9,세야스시,10~20만원대,12:00~15:00/18:00~22:00,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,가능,"런치오마카세(10만원), 디너오마카세(20만원)",월요일 휴무


In [ ]:
df_bluer.to_csv('블루리본서베이.csv', index=False)